<a href="https://colab.research.google.com/github/GaiMarina/BlockOneFinalTest/blob/main/7_AB_HomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

На основе выбранных продуктов из первого вебинара, выберите 2 A/B-теста и составьте план эксперимента, чтобы избежать Peeking Problem

Распишите, какие метрики и как вы будете анализировать

---

In [2]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin 
import numpy as np
import pandas as pd

# Дизайн эксперимента 1. 
## Тестируем две версии сайта https://www.flightradar24.com/
1.	### Гипотеза.
Если на стартовую страницу добавить рекламный баннер с предложением узнать номер рейса и маршрут как раз сейчас пролетающего над пользователем самолета простым наведением камеры телефона, это увеличит процент переходов в магазин для скачивания приложения Flightradar (CTR) на 5% и количество установок на клик (CTI) на 2 %, т.к. баннер будет привлекательным для клиента, а предполагаемый опыт достаточно любопытным для того, чтобы его испытать.
2.	### Что делаем.
Контрольная версия: оставляем текущий вид страницы.
Тестовая версия: добавляем кнопки Download on the App Store и Get it on Google Play на баннер 4 x 6 см с сохранением текущего месторасположения кнопок.
3.	### На каких пользователях тестируем.
На всех пользователях.
4.	### Метрики: 
СTI - primary, CTR - secondary
5.	### План действий.
Если наш эксперимент будет положительным и мы зафиксируем ожидаемое улучшение в ключевых метриках, то масштабируем изменение и ""выкатываем"" его на всех пользователей. 
Если основные показатели метрик падают либо просто не изменяются, ""откатываем"" эксперимент.

## CTI
---
Число установок приложения на количество кликов по рекламному банеру.

|                | **Control** | **Test** |
|:--------------:|:------------:|:------------:|
|   **Traffic_mean**  |     5000     |     5000     |
| **downloads/clicks** |      30%     |      31.5%     |

In [19]:
prob_1, prob_2 = 0.3, 0.3 * 1.05
traffic_mean = 5_000

In [20]:
#Расчёт effect_size для пропорций

def calc_proportion_effect_size(conv_1: int, conv_2: int):
    es_formula = 2  * asin(np.sqrt(conv_1)) - 2 * asin(np.sqrt(conv_2))
    es_import = proportion_effectsize(conv_1, conv_2)
    
    return es_formula, es_import

In [21]:
#Расчёт минимально необходимой выборки * 2(для теста и контроля) для пропорций

def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

In [26]:
es = calc_proportion_effect_size(prob_1, prob_2)[0]
print(f'Необходимое количество наблюдений для 2-х групп: {calc_sample_size_proportion(es)}\n'
      f'Необходимое количество дней при alpha = 5% и beta = 20%: {round(((calc_sample_size_proportion(es)) / traffic_mean), 2)}')

Необходимое количество наблюдений для 2-х групп: 29709
Необходимое количество дней при alpha = 5% и beta = 20%: 5.94


## CTR
Показатель CTR – это отношение числа посетителей страницы, перешедших по рекламе к суммарному числу посетителей за некий определённый период времени.

**Метрика пропорциональная.**

|                | **Control** | **Test** |
|:--------------:|:------------:|:------------:|
|   **Traffic_mean**  |     5000     |     5000     |
| **clicks/comers** |      14%     |      14.28%     |


In [24]:
transs_1, transs_2 = 0.14, 0.14 * 1.02
mean_traffic = 5_000

In [27]:
e_s = calc_proportion_effect_size(transs_1, transs_2)[0]
print(f'Необходимое количество наблюдений для 2-х групп: {calc_sample_size_proportion(e_s)}\n'
      f'Необходимое количество дней при alpha = 5% и beta = 20%: {round(((calc_sample_size_proportion(es)) / mean_traffic), 2)}')

Необходимое количество наблюдений для 2-х групп: 486162
Необходимое количество дней при alpha = 5% и beta = 20%: 5.94


---

Если наш эксперимент будет положительным, и мы зафиксируем ожидаемое улучшение в CTI и не "просядет" CTR, то масштабируем изменение и «выкатываем» его на всех пользователей.

Расчетное время проведения эксперимента - 6 дней. Рекомендовано воздержаться от принятия каких либо решений до истечения данной длительности эксперимента во избежание peeking problem.

# Дизайн эксперимента 2.

1.	### Гипотеза.
3. Гипотеза, что, если добавить предупреждение "You won't be charged after your 7 days trial ends" к предложению попробовать 7 дневный бесплатный пробный период применительно к любому тарифу-подписке, сам по себе, увеличит метрику ARPPU на 1%, т.к. побудит пользователей свободнее пользоваться всеми бонусами бесплатного периода и испытать на себе все плюсы платной подписки при нулевом риске быть подключенным на платный вариант без согласия пользователя в дальнейшем.
2.	### Что делаем.
Контрольная версия: оставляем текущий вид страницы с платными тарифами.
Тестовая версия: добавляем предупреждение "You won't be charged after your 7 days trial ends" к предложению попробовать 7 дневный бесплатный пробный период применительно к любому тарифу-подписке.
3.	### На каких пользователях тестируем.
На всех пользователях.
4.	### Метрики: 
ARPPU
5.	### План действий.
В случае, если показатели ARPPU не просядут, "выкатываем" эксперимент на всех пользователей.

---

ARPPU = Платежи LT/Впервые заплативших LT

**Метрика непрерывная.**

|                | **Control** | **Test** |
|:--------------:|:------------:|:------------:|
|   **average**  |     5200     |   5250       |
| **SD** |      70     |      80.5    |

In [79]:
mu_control, mu_test = 5_200, 5_250
std_control, std_test = 3_500, 4_000          # 35, 40
mean_traffic = 5_000

In [80]:
#Расчёт effect_size для непрерывных метрик

def calc_continuous_effect_size(mean_1: Union[float, int],
                                std_1: Union[float, int],
                                mean_2: Union[float, int],
                                std_2: Union[float, int]):
    
    es_formula = (mean_1 - mean_2) / ((std_1**2 + std_2**2) / 2) ** 0.5
    
    return es_formula

In [81]:
effect_size = calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test)
effect_size

-0.013303802104754786

In [82]:
# Расчёт минимально необходимой выборки * 2(для теста и контроля) для непрерывной метрики

def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

In [85]:
sample_size = calc_sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)
print(f'Необходимое количество наблюдений для 2-х групп: {sample_size}\n'
      f'При тестировании на всех пользователях время, необходимое для проведения теста, составит {(sample_size / mean_traffic)} дней.')

Необходимое количество наблюдений для 2-х групп: 177386
При тестировании на всех пользователях время, необходимое для проведения теста, составит 35.4772 дней.
